Goal: separate the ingrdients in the FoodKG table into a list of ingredients

In [1]:
# !pip install --upgrade pip -q
# !pip install psycopg2-binary -q
# !pip install SQLAlchemy -q
# !pip install spacy -q
# !python -m spacy download en_core_web_sm -q

In [ ]:
import json
import os

import pandas as pd
from dotenv import load_dotenv

from llm_util import process_food_kg_df, assemble_food_kg_df

# from sqlalchemy import create_engine

load_dotenv()  # Take environment variables from .env
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1",  # local model
    api_key=api_key
)

In [ ]:
# # Credentials
# host = os.getenv("POSTGRES_HOST")
# port = os.getenv("POSTGRES_PORT")
# database = os.getenv("POSTGRES_DATABASE")
# user = os.getenv("POSTGRES_USER")
# password = os.getenv("POSTGRES_PASSWORD")
# # Create the connection engine
# engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")
# # Test the connection
# try:
#     with engine.connect() as conn:
#         print("✅ Connected successfully!")
# except Exception as e:
#     print("❌ Connection failed:", e)

In [4]:
# # Test: query 5 rows
# query = "SELECT * FROM \"FoodKG\" LIMIT 5;"  # quotes protect uppercase table names
# df = pd.read_sql(query, engine)
# # display nicely
# pd.set_option('display.max_columns', None)  # show all columns
# pd.set_option('display.max_rows', 5)  # show up to 5 rows
# display(df)

In [5]:
df = pd.read_csv("data/raw/FoodKG_ingredients.csv")

In [6]:
df.head()

,id,title,ingredients
0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva..."
1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ..."
2,2,Creamy Corn,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg..."
3,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans..."
4,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ..."


In [ ]:
model = "qwen/qwen3-4b-2507"
# model="openai/gpt-oss-20b",

process_food_kg_df(
    df=df,
    model=model,
    client=client,
    restart=True,  # reset the checkpoints
    stop_at=1000
)

In [ ]:
final_df = assemble_food_kg_df()

In [ ]:
len(final_df)

In [ ]:
# CSV can’t store native Python lists (they are objects), so use JSON dump
ingredients_table = pd.DataFrame({
    "id": final_df["id"],
    "title": final_df["title"],
    "ingredients": final_df["ingredients"],  # new, kept upon request
    "ingredients_normalized": final_df["ingredients_normalized"].apply(json.dumps),
}).sort_values("id")
# Create a directory if it doesn't exist and save

In [ ]:
os.makedirs("data/output", exist_ok=True)
ingredients_table.to_csv("data/output/processed_ingredients.csv", index=False)

In [ ]:
ingredients_table.tail()

In [ ]:
final_df.head()